In [1]:
import datetime
import heapq
import sqlite3

In [2]:
TIME_FORMAT="%Y-%m-%d-%H%M"
ONE_DAY=86400
ONE_HOUR=3600
ONE_MINUTE=60
DB_PATH='things.db'

In [3]:
counter = 0

In [4]:
class Task():
    def __init__(self, name="Untitled-Task-{}".format(datetime.datetime.today().strftime(TIME_FORMAT)), 
                period=86400, 
                history=[datetime.datetime.now()],
                comment="",
                category="", thingID=None):
        self.name = name
        self.period = period
        self.history=history
        self.due =  history[-1] + datetime.timedelta(seconds=period)
        self.comment = comment
        self.category = category
        global counter
        add = False
        if thingID is None:
            add = True
            thingID = counter
            counter += 1
        elif thingID >= counter:
            counter = thingID + 1
        self.thingID = thingID
        print(self.thingID, self.name, self.period, self.due, self.comment, self.category)
        if add:
            add_new_task_to_DB(self, path=DB_PATH)
    
    def check(self, time=datetime.datetime.now(), *otherTimes):
        if isinstance(time, datetime.datetime):
            self.history.append(time)
        elif isinstance(time, tuple):
            try:
                y, m, d = time
                time = datetime.datetime(y,m,d)
                self.history.append(time)
            except:
                print("Could not parse time of {}".format(time))
        latest = time
        if len(otherTimes) > 0:
            for timestamp in otherTimes:
                if isinstance(timestamp, datetime.datetime):
                    self.history.append(timestamp)
                    if timestamp > time: 
                        latest = timestamp
                elif isinstance(timestamp, tuple):
                    try:
                        y, m, d = timestamp
                        timestamp = datetime.datetime(y,m,d)
                        self.history.append(timestamp)
                        if timestamp > time:
                            latest = timestamp
                    except:
                        print("Could not parse time of {}".format(timestamp))
        newdue = latest + datetime.timedelta(seconds=self.period)
        if newdue > self.due:
            self.due = newdue
        
    
    def get_period(self,):
        return self.period
    
    def print_period(self, ):
        return format_period(self.get_period())

    def set_period(self, new_period):
        self.period = new_period
    
    def get_due(self):
        return self.due
    
    def print_due(self):
        return format_time(self.get_due())

    def set_due(self, new_due):
        self.due = new_due      

    def get_last(self):
        return self.history[-1]

    def print_last(self):
        return format_time(self.get_last())

    def get_name(self, ):
        return self.name

    def set_name(self, new_name):
        self.name = new_name      

    def get_comment(self, ):
        return self.comment

    def set_comment(self, new_comment):
        self.comment = new_comment     

    def get_category(self, ):
        return self.category

    def set_category(self, new_category):
        self.category = new_category    

    def get_thingID(self, ):
        return self.thingID

    def set_thingID(self, new_thingID):
        self.thingID = new_thingID

In [5]:
class TaskList():
    def __init__(self, taskList=[]):
        self.tasks = taskList
        self.dict = {task.thingID:task for task in taskList}

    def get_all(self):
        return self.tasks

    def get_task(self, query):
        if isinstance(query, str):
            print("STRING", query)
            for task in self.tasks:
                if task.name.lower().strip() == query.lower().strip():
                    return task
            print("task not found!")
            return None
        elif isinstance(query, int):
            thistask = self.dict.get(query)
            if thistask:
                return thistask
            else:
                print("task not found!")
                return None
    
    def show(self, ):
        heap = []
        for task in self.tasks:
            #print(task.name, task.due, task.print_period())
            tilldue = task.due - datetime.datetime.now()
            sec = int(tilldue.total_seconds())
            #print(tilldue)
            heapq.heappush(heap, (sec, task.name))
        returnlist = []
        heap.sort()
        for x in heap:
            name = x[1]
            due = x[0]
            due = format_period(due)
            returnlist.append("{}\t\t{}".format(name, due))
        print("\n".join(returnlist))
    
    def add(self, name="Untitled-Task-{}".format(datetime.datetime.today().strftime(TIME_FORMAT)), period=86400, last=datetime.datetime.now()):
        newTask = Task(name, period, last)
        thingID = newTask.thingID
        self.tasks.append(newTask)
        self.dict[thingID] = newTask

    def check(self, query, time=datetime.datetime.now(), *otherTimes): 
        task = self.get_task(query)
        if task:
            if len(otherTimes) > 0:
                for timestamp in otherTimes:
                    if isinstance(timestamp, datetime.datetime):
                        task.check(timestamp)
                    elif isinstance(timestamp, tuple):
                        try:
                            y, m, d = timestamp
                            timestamp = datetime.datetime(y,m,d)
                            task.check(timestamp)
                        except:
                            print("Could not parse time of {}".format(timestamp))
            if isinstance(time, datetime.datetime):
                task.check(time)
            elif isinstance(time, tuple):
                try:
                    y, m, d = time
                    time = datetime.datetime(y,m,d)
                    task.check(time)
                except:
                    print("Could not parse time of {}".format(time))
        else:
            print("task {} not found.".format(task))

    def update_to_DB(self):
        update_all_to_DB(self.get_all(), DB_PATH)

In [6]:
def read_db(path='things.db'):
    db = path
    conn = sqlite3.connect(db)
    c = conn.cursor()
    things = c.execute('SELECT * FROM things')
    global counter
    conn.close()
    return things

def new_task_list_from_db(path='things.db'):
    global counter
    db = path
    conn = sqlite3.connect(db)
    c = conn.cursor()
    things = c.execute('SELECT * FROM things')
    objectified_things = []
    maxID = 0
    for thing in things:
        #print(thing)
        thingID, name, period, history, comment, category = thing
        if thingID > maxID:
            maxID = thingID
        history = sorted(set([datetime.datetime.strptime(x, TIME_FORMAT) for x in history.split(", ")]))
        #print(name, period, history, comment, category)
        mytask = Task(name, period, history, comment, category, thingID)
        objectified_things.append(mytask)
    if maxID > counter:
        counter = maxID + 1
    newlist = TaskList(objectified_things)
    conn.close()
    return newlist

In [7]:
def update_all_to_DB(things, path='things.db'): 
    db = path
    conn = sqlite3.connect(db)
    c = conn.cursor()
    mycommand = 'SELECT id FROM things;'
    thingIDs = [row[0] for row in c.execute(mycommand)]
    for thing in things:
        if thing.thingID in thingIDs:
            print(thing.name)
            mycommand = "UPDATE things set name = '{}', period = {}, history = '{}', comment = '{}', category = '{}' WHERE id = {};".format(thing.name, thing.period, ", ".join([x.strftime(TIME_FORMAT) for x in thing.history]), thing.comment, thing.category, thing.thingID)
            print(mycommand)
            c.execute(mycommand)
        else:
            print("ThingID {} not in DB! Adding...".format(thing.thingID))
            mycommand = "INSERT INTO things VALUES ({}, '{}', {}, '{}', '{}', '{}');".format(thing.thingID, thing.name, thing.period, ", ".join([x.strftime(TIME_FORMAT) for x in thing.history]), thing.comment, thing.category)
            print(mycommand)
            c.execute(mycommand)
        conn.commit()
    conn.close()

def add_new_task_to_DB(thing, path='things.db'): 
    db = path
    conn = sqlite3.connect(db)
    c = conn.cursor()
    mycommand = "INSERT INTO things VALUES ({}, '{}', {}, '{}', '{}', '{}');".format(thing.thingID, thing.name, thing.period, ", ".join([x.strftime(TIME_FORMAT) for x in thing.history]), thing.comment, thing.category)
    print(mycommand)
    c.execute(mycommand)
    conn.commit()
    conn.close()

In [8]:
def format_time(time):
    if isinstance(time, datetime.datetime):
        return time.strftime(TIME_FORMAT)
    else:
        print("Please pass a datetime object! I can't work under these conditions!!")


def format_period(period):
    def format_plural(n, unit):
        if n == 1:
            return "{} {}".format(n, unit)
        else:
            return "{} {}s".format(n, unit)
    return_str = ""
    if period < 0:
        return "OVERDUE"
    elif period > ONE_DAY:
        days = int(period/ONE_DAY)
        hours = int((period%ONE_DAY)/ONE_HOUR)
        return_str += format_plural(days, "day")
        if hours > 0:
            return_str += ", {}".format(format_plural(hours, "hour"))
        return return_str
    else:
        hours = int(period/ONE_HOUR)
        minutes = int((period%ONE_HOUR)/ONE_MINUTE)
        return_str += format_plural(hours, "hour")
        if minutes > 0:
            return_str += ", {}".format(format_plural(minutes, "minute"))
        return return_str

In [9]:
newlist = new_task_list_from_db()

0 Dolores 1296000 2020-10-29 18:36:00  
1 Jaqen 1296000 2020-10-21 17:00:00  
2 Elan 864000 2020-10-16 17:00:00  
3 Dishes 86400 2020-10-07 17:38:00  
4 Pilates 345600 2020-10-14 00:00:00  
5 Yoga 345600 2020-10-17 00:00:00  
6 Running 345600 2020-10-15 00:00:00  
7 Bodyweight 345600 2020-10-08 12:15:00  
8 Lemon Tree 864000 2020-10-16 12:00:00  
9 Backup 7776000 2021-01-02 12:07:00  
10 Sheets 2592000 2020-10-12 12:00:00  
11 Adventure 604800 2020-10-19 00:00:00  
12 Nails 604800 2020-10-11 15:00:00  
13 Laundry 604800 2020-10-11 12:00:00  


In [56]:
#newlist.get_task('pilates').set_period(ONE_DAY)
newlist.get_task('yoga').get_due()

STRING yoga


datetime.datetime(2020, 10, 10, 8, 0)

datetime.datetime(2020, 10, 21, 17, 0)

In [24]:
#newlist.check('running', datetime.datetime(2020, 10, 15, 8, 15))
newlist.check('lemon tree')


STRING lemon tree


In [21]:
# newlist.check('dishes', time=datetime.datetime(2020,10,5,22))
# newlist.check('yoga', time=datetime.datetime(2020,10,3,8))
# newlist.check('yoga', time=datetime.datetime(2020,10,6,8))
# newlist.check('running', time=datetime.datetime(2020,10,4,13))
# newlist.check('pilates', time=datetime.datetime(2020,10,4,9))
# newlist.check('lemon tree', time=datetime.datetime(2020,10,6,12))
# newlist.check('laundry', time=datetime.datetime(2020,10,4,12))
# newlist.check('nails', time=datetime.datetime(2020,10,4,15))
#newlist.check('dolores')
#newlist.check('jaqen')
newlist.check('elan')


STRING elan


In [22]:
newlist.show()

Dishes		OVERDUE
Bodyweight		OVERDUE
Laundry		OVERDUE
Nails		OVERDUE
Sheets		OVERDUE
Pilates		OVERDUE
Yoga		1 day, 10 hours
Adventure		3 days, 10 hours
Running		3 days, 18 hours
Dolores		6 days, 3 hours
Jaqen		6 days, 3 hours
Lemon Tree		9 days, 3 hours
Elan		9 days, 23 hours
Backup		78 days, 22 hours


In [75]:
newlist.check('adventure', (2020,10,12))

STRING adventure


In [21]:
# conn = sqlite3.connect('things.db')
# c = conn.cursor()

# mycommand = 'SELECT * FROM things WHERE id = 10;'
# for row in c.execute(mycommand):
#     print(row)
# conn.close()

In [10]:
# dolores=Task("Dolores", ONE_DAY * 15, [datetime.datetime(2020,9,25,15)])
# jaqen=Task("Jaqen", ONE_DAY * 15, [datetime.datetime(2020,9,16,15)])
# elan=Task("Elan", ONE_DAY * 10, [datetime.datetime(2020,9,27,15)])
# dishes=Task("Dishes", ONE_DAY - 100, [datetime.datetime.now() - datetime.timedelta(days=2)])
# pilates=Task("Pilates", ONE_DAY * 4, [datetime.datetime(2020,9,25,11), datetime.datetime(2020,10,3,10)])
# yoga=Task("Yoga", ONE_DAY * 4, [datetime.datetime(2020,9,27,11),datetime.datetime(2020,10,4,10)])
# running=Task("Running", ONE_DAY * 4, [datetime.datetime(2020,9,28,8), datetime.datetime(2020,10,2,7)])
# bodyweight=Task("Bodyweight", ONE_DAY * 4, [datetime.datetime(2020,9,25,17)])
# zhou=Task("Lemon Tree", ONE_DAY * 10, [datetime.datetime(2020,10,1,10)])
# backup=Task("Backup", ONE_DAY * 90)
# sheets=Task("Sheets", ONE_DAY * 30, [datetime.datetime(2020,9,12,12)])
# adventure=Task("Adventure", ONE_DAY * 7, [datetime.datetime(2020,9,27,15),datetime.datetime(2020,10,3,18)])
# nails=Task("Nails", ONE_DAY * 7, [datetime.datetime(2020,10,4,10)])
# laundry=Task("Laundry", ONE_DAY * 7, [datetime.datetime(2020,9,26,10),datetime.datetime(2020,10,1,10)])

0 Dolores 1296000 2020-10-10 15:00:00  
INSERT INTO things VALUES (0, 'Dolores', 1296000, '2020-09-25-1500', '', '');
1 Jaqen 1296000 2020-10-01 15:00:00  
INSERT INTO things VALUES (1, 'Jaqen', 1296000, '2020-09-16-1500', '', '');
2 Elan 864000 2020-10-07 15:00:00  
INSERT INTO things VALUES (2, 'Elan', 864000, '2020-09-27-1500', '', '');
3 Dishes 86300 2020-10-03 12:05:40.579614  
INSERT INTO things VALUES (3, 'Dishes', 86300, '2020-10-02-1207', '', '');
4 Pilates 345600 2020-10-07 10:00:00  
INSERT INTO things VALUES (4, 'Pilates', 345600, '2020-09-25-1100, 2020-10-03-1000', '', '');
5 Yoga 345600 2020-10-08 10:00:00  
INSERT INTO things VALUES (5, 'Yoga', 345600, '2020-09-27-1100, 2020-10-04-1000', '', '');
6 Running 345600 2020-10-06 07:00:00  
INSERT INTO things VALUES (6, 'Running', 345600, '2020-09-28-0800, 2020-10-02-0700', '', '');
7 Bodyweight 345600 2020-09-29 17:00:00  
INSERT INTO things VALUES (7, 'Bodyweight', 345600, '2020-09-25-1700', '', '');
8 Lemon Tree 864000 2020

In [11]:
# def save_db(things, path='things.db'): # TODO: UPDATE instead of INSERT
#     db = path
#     conn = sqlite3.connect(db)
#     c = conn.cursor()
#     for thing in things:
#         print(thing.name)
#         mycommand = "INSERT INTO things VALUES ({}, '{}', {}, '{}', '{}', '{}')".format(thing.thingID, thing.name, thing.period, ", ".join([x.strftime(TIME_FORMAT) for x in thing.history]), thing.comment, thing.category)
#         print(mycommand)
#         c.execute(mycommand)
#     conn.commit()
#     conn.close()

In [12]:
# mytasks = [dolores, jaqen, elan, dishes, pilates, yoga, running, bodyweight, zhou, backup, sheets, adventure, nails, laundry]
# for task in mytasks:
#     print(task.name, task.print_period(), task.due)

Dolores 15 days 2020-10-10 15:00:00
Jaqen 15 days 2020-10-01 15:00:00
Elan 10 days 2020-10-07 15:00:00
Dishes 23 hours, 58 minutes 2020-10-03 12:05:40.579614
Pilates 4 days 2020-10-07 10:00:00
Yoga 4 days 2020-10-08 10:00:00
Running 4 days 2020-10-06 07:00:00
Bodyweight 4 days 2020-09-29 17:00:00
Lemon Tree 10 days 2020-10-11 10:00:00
Backup 90 days 2021-01-02 12:07:02.449221
Sheets 30 days 2020-10-12 12:00:00
Adventure 7 days 2020-10-10 18:00:00
Nails 7 days 2020-10-11 10:00:00
Laundry 7 days 2020-10-08 10:00:00


In [13]:
mylist = TaskList(mytasks)

In [17]:
# save_db(mylist.get_all(), DB_PATH)

In [18]:
newlist.show()

Jaqen		OVERDUE
Dishes		4 hours, 51 minutes
Elan		21 hours, 53 minutes
Pilates		1 day, 15 hours
Bodyweight		1 day, 19 hours
Running		1 day, 19 hours
Yoga		3 days, 14 hours
Dolores		3 days, 21 hours
Adventure		4 days
Laundry		4 days, 18 hours
Nails		4 days, 21 hours
Sheets		5 days, 18 hours
Lemon Tree		9 days, 18 hours
Backup		87 days, 19 hours


In [25]:
update_all_to_DB(newlist.get_all(), DB_PATH)

Dolores
UPDATE things set name = 'Dolores', period = 1296000, history = '2020-09-25-1500, 2020-10-06-1700', comment = '', category = '' WHERE id = 0;
Jaqen
UPDATE things set name = 'Jaqen', period = 1296000, history = '2020-09-16-1500, 2020-10-06-1700', comment = '', category = '' WHERE id = 1;
Elan
UPDATE things set name = 'Elan', period = 864000, history = '2020-09-27-1500, 2020-10-06-1700, 2020-10-15-1343', comment = '', category = '' WHERE id = 2;
Dishes
UPDATE things set name = 'Dishes', period = 86400, history = '2020-10-02-1207, 2020-10-04-1211, 2020-10-05-2200, 2020-10-06-1738', comment = '', category = '' WHERE id = 3;
Pilates
UPDATE things set name = 'Pilates', period = 345600, history = '2020-09-25-1100, 2020-10-03-1000, 2020-10-04-0900, 2020-10-10-0000', comment = '', category = '' WHERE id = 4;
Yoga
UPDATE things set name = 'Yoga', period = 345600, history = '2020-09-27-1100, 2020-10-03-0800, 2020-10-04-1000, 2020-10-06-0800, 2020-10-09-0000, 2020-10-13-0000', comment = ''